In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from basic_tools import *

debug=False

# (Optional) Automatically make conversion manual

## Load codebook

In [2]:
codebook_AS=pd.read_csv(codebook_AS_path)#,header=1)
codebook_CT=pd.read_csv(codebook_CT_path)#,header=1)
codebook_NC=pd.read_csv(codebook_NC_path)

codebook_column_dict={"테이블명(영문)":"table_code","변수명": "phenotype_code","변수설명":"description", "변수값(코드) 설명": "value_description","변수유형":"type","설문지":"questionnaire"}
codebook_AS=codebook_AS.rename(columns=codebook_column_dict)
codebook_CT=codebook_CT.rename(columns=codebook_column_dict)
codebook_NC=codebook_NC.rename(columns=codebook_column_dict)

codebook_AS['type']=codebook_AS['type'].replace('범주형','categorical').replace('연속형','continuous').replace('서술형','descriptive').replace('날짜','date')
codebook_CT['type']=codebook_CT['type'].replace('범주형','categorical').replace('연속형','continuous').replace('서술형','descriptive').replace('날짜','date')
codebook_NC['type']=codebook_NC['type'].replace('범주형','categorical').replace('연속형','continuous').replace('서술형','descriptive').replace('날짜','date')

codebook_AS['phenotype_code']=codebook_AS['phenotype_code'].str.upper()

codebook_total=pd.concat([codebook_AS,codebook_CT,codebook_NC])

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [3]:
"""
def codebook_selector(cohort):
    if cohort=='AS':
        return codebook_AS
    elif cohort=='CT':
        return codebook_CT
    elif cohort=='NC':
        return codebook_NC
    else:
        raise
"""

"\ndef codebook_selector(cohort):\n    if cohort=='AS':\n        return codebook_AS\n    elif cohort=='CT':\n        return codebook_CT\n    elif cohort=='NC':\n        return codebook_NC\n    else:\n        raise\n"

In [4]:
"""
codebook_custommerge=pd.read_csv(codebook_custommerge_path)
codebook_custommerge
codebook_custommerge=codebook_custommerge.rename(columns=codebook_column_dict)
codebook_custommerge['phenotype_code']=codebook_custommerge['phenotype_code'].str.upper()
"""

"\ncodebook_custommerge=pd.read_csv(codebook_custommerge_path)\ncodebook_custommerge\ncodebook_custommerge=codebook_custommerge.rename(columns=codebook_column_dict)\ncodebook_custommerge['phenotype_code']=codebook_custommerge['phenotype_code'].str.upper()\n"

In [20]:
codebook_total_temp=codebook_total.copy()
codebook_total_temp=codebook_total_temp.applymap(lambda x: replace_characters(x,{'\n':'',' ':'','■':'',',':''}))

In [19]:
conversion_manual=pd.DataFrame(columns=['phenotype_code','commands','custom_id'])
conversion_manual.set_index('phenotype_code',inplace=True)

In [21]:
for idx,row in codebook_total_temp.iterrows():
    try:
        phenotype_raw_series=phenotype_raw[row['phenotype_code']]
    except KeyError:
        continue
    
    #ignore environmental 
    if '음주' in row['description'] or '빈도' in row['description']:
        continue
    
    #set new phenotype name
    phenotype_name=row['phenotype_code'][4:]
    try:
        phenotype_name_temp=codebook_merged.set_index('phenotype_code')['trait_name'][row['phenotype_code']]
        #print(phenotype_name_temp,'here')
        #print(phenotype_name_temp,type(phenotype_name_temp),np.isnan(phenotype_name_temp))
        if type(phenotype_name_temp)==str:
            phenotype_name=phenotype_name_temp
    except:
        pass
    
    phenotype_name=phenotype_name.replace(' ','_')
    
    if row['type']=='categorical':
        #print(row['value_description'])
        if row['value_description']=='1=아니오2=예':
            conversion_manual.loc[row['phenotype_code']]='{}@x==1@1|{}@x==2@2'.format(phenotype_name,phenotype_name)
        elif row['value_description']=='0=normal1=abnormal':
            conversion_manual.loc[row['phenotype_code']]='{}@x==0@1|{}@x==1@2'.format(phenotype_name,phenotype_name)
        else:
            #print(row['description'],row['value_description'])
            pass
        
    elif row['type']=='continuous':
        conversion_manual.loc[row['phenotype_code']]='{}@(x!=66666)&(x!=77777)&(x!=99999)@x'.format(phenotype_name)

In [22]:
conversion_manual['custom_id']=codebook_merged.set_index('phenotype_code')['일련번호merged'].loc[conversion_manual.index.values]
#conversion_manual.sort_values('custom_id').to_csv(conversion_manual_path,sep='\t')

# (deprecated) Step1. duplicated phenotype code

## Group variable of common phenotype code

In [7]:
phenotype_raw_columns=phenotype_raw.columns.to_series()
phenotype_raw_columns[phenotype_raw_columns.index]=phenotype_raw_columns.index.str.slice(start=4)
phenotype_raw_columns=phenotype_raw_columns[phenotype_raw_columns.duplicated(keep=False)]
phenotype_groups=phenotype_raw_columns.groupby(phenotype_raw_columns).groups